In [0]:
%sql
USE CATALOG medalhao;
CREATE SCHEMA IF NOT EXISTS gold;
USE SCHEMA gold;

In [0]:
from pyspark.sql.window import Window
from datetime import datetime, timedelta
from pyspark.sql.types import DateType, DecimalType
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.functions import col, to_date, lead, sequence, explode, year, quarter, month, weekofyear, dayofmonth, dayofweek, date_format, when, lit

# definicao do caminho do schema
catalogo = 'medalhao'
silver_db_name = 'silver'
gold_db_name = 'gold'

spark = SparkSession.builder.getOrCreate()

In [0]:
# 1.1 tabela gold.ft_vendas_consumidor_local

df_ft_consumidores = spark.table(f'{catalogo}.{silver_db_name}.ft_consumidores')
df_ft_pedido_total = spark.table(f'{catalogo}.{silver_db_name}.ft_pedido_total')

# adicionando coluna status para checar pedidos entregues ou enviados 

df_ft_vendas_consumidor_local = (
    df_ft_consumidores.join(
        df_ft_pedido_total,
        on='id_consumidor',
        how='inner'
    )
    .select(
        col('id_pedido').cast('string'),
        col('id_consumidor').cast('string'),
        col('valor_total_pago_brl').cast(DecimalType(12,2)).alias('valor_total_pedido_brl'),
        col('cidade').cast('string'),
        col('estado').cast('string'),
        to_date(col('data_pedido')).alias('data_pedido'),
        col('status')
    )
)

df_ft_vendas_consumidor_local.write.mode('overwrite').saveAsTable(f'{catalogo}.{gold_db_name}.ft_vendas_consumidor_local')

In [0]:
# 1.2 view gold.view_total_compras_por_consumidor

# levando em conta somente os pedidos entregues ou enviados 
query = """
CREATE OR REPLACE VIEW gold.view_total_compras_por_consumidor AS
SELECT
  cidade,
  estado,
  COUNT(id_pedido) AS quantidade_vendas,
  SUM(valor_total_pedido_brl) AS valor_total_localidade
FROM gold.ft_vendas_consumidor_local
WHERE status IN ('entregue', 'enviado')
GROUP BY cidade, estado
"""

spark.sql(query)

display(spark.sql("SELECT * FROM gold.view_total_compras_por_consumidor"))

In [0]:
# 1.2.1 query total de vendas por estado

# levando em conta somente os pedidos entregues ou enviados 

display(spark.sql("""
SELECT
  estado,
  SUM(valor_total_localidade) AS total_vendas_estado
FROM gold.view_total_compras_por_consumidor
WHERE estado IN (
  SELECT estado
  FROM gold.ft_vendas_consumidor_local
)
GROUP BY estado
ORDER BY total_vendas_estado DESC
"""))

In [0]:
# 2.1 tabela gold.ft_atrasos_pedidos_local_vendedor
df_ft_pedidos = spark.table(f'{catalogo}.{silver_db_name}.ft_pedidos')
df_ft_consumidores = spark.table(f'{catalogo}.{silver_db_name}.ft_consumidores')
df_ft_itens_pedidos = spark.table(f'{catalogo}.{silver_db_name}.ft_itens_pedidos')

df_ft_atrasos_pedidos_local_vendedor = (
    df_ft_pedidos
    .join(df_ft_consumidores, on='id_consumidor', how='inner')
    .join(df_ft_itens_pedidos, on='id_pedido', how='inner')
    .select(
        col('id_pedido'),
        col('id_vendedor'),
        col('id_consumidor'),
        col('entrega_no_prazo'),
        col('tempo_entrega_dias'),
        col('tempo_entrega_estimado_dias'),
        col('cidade'),
        col('estado')
    )
)

df_ft_atrasos_pedidos_local_vendedor.limit(20).display()

df_ft_atrasos_pedidos_local_vendedor.write.mode('overwrite').saveAsTable(f'{catalogo}.{gold_db_name}.ft_atrasos_pedidos_local_vendedor')

In [0]:
# 2.2.1 view gold.view_tempo_medio_entrega_localidade
query = """
CREATE OR REPLACE VIEW gold.view_tempo_medio_entrega_localidade AS
SELECT
  cidade,
  estado,
  AVG(tempo_entrega_dias) AS tempo_medio_entrega,
  AVG(tempo_entrega_estimado_dias) AS tempo_medio_estimado,
  CASE
    WHEN AVG(tempo_entrega_dias) > AVG(tempo_entrega_estimado_dias) THEN 'SIM'
    ELSE 'NÃO'
  END AS entrega_maior_que_estimado
FROM gold.ft_atrasos_pedidos_local_vendedor
GROUP BY cidade, estado
"""

spark.sql(query)

display(spark.sql("SELECT * FROM gold.view_tempo_medio_entrega_localidade").limit(5))

In [0]:
# 2.2.2 view gold.view_vendedor_pontualidade
query = """
CREATE OR REPLACE VIEW gold.view_vendedor_pontualidade AS
SELECT
  id_vendedor,
  COUNT(id_pedido) AS total_pedidos,
  SUM(CASE WHEN entrega_no_prazo = 'Não' THEN 1 ELSE 0 END) AS total_atrasados,
  ROUND(100.0 * SUM(CASE WHEN entrega_no_prazo = 'Não' THEN 1 ELSE 0 END) / COUNT(id_pedido), 2) AS percentual_atraso
FROM gold.ft_atrasos_pedidos_local_vendedor
GROUP BY id_vendedor
"""

spark.sql(query)

display(spark.sql("SELECT * FROM gold.view_vendedor_pontualidade"))

In [0]:
# 3.1 tabela gold.dm_tempo

# Definir datas de início e fim
data_inicio = '2016-01-01'
data_fim = '2019-01-01'

df_datas = (
    spark.createDataFrame([(data_inicio, data_fim)], ['data_inicio', 'data_fim'])
    .select(explode(sequence(col('data_inicio').cast(DateType()), col('data_fim').cast(DateType()))).alias('sk_tempo'))
    .withColumn('ano', year(col('sk_tempo')))
    .withColumn('trimestre', quarter(col('sk_tempo')))
    .withColumn('mes', month(col('sk_tempo')))
    .withColumn('semana_do_ano', weekofyear(col('sk_tempo')))
    .withColumn('dia', dayofmonth(col('sk_tempo')))
    .withColumn('dia_da_semana_num', dayofweek(col('sk_tempo')))
    .withColumn('dia_da_semana_nome', 
        when(col('dia_da_semana_num') == 1, 'Domingo')
        .when(col('dia_da_semana_num') == 2, 'Segunda-feira')
        .when(col('dia_da_semana_num') == 3, 'Terça-feira')
        .when(col('dia_da_semana_num') == 4, 'Quarta-feira')
        .when(col('dia_da_semana_num') == 5, 'Quinta-feira')
        .when(col('dia_da_semana_num') == 6, 'Sexta-feira')
        .when(col('dia_da_semana_num') == 7, 'Sábado')
    )
    .withColumn('mes_nome',
        when(col('mes') == 1, 'Janeiro')
        .when(col('mes') == 2, 'Fevereiro')
        .when(col('mes') == 3, 'Março')
        .when(col('mes') == 4, 'Abril')
        .when(col('mes') == 5, 'Maio')
        .when(col('mes') == 6, 'Junho')
        .when(col('mes') == 7, 'Julho')
        .when(col('mes') == 8, 'Agosto')
        .when(col('mes') == 9, 'Setembro')
        .when(col('mes') == 10, 'Outubro')
        .when(col('mes') == 11, 'Novembro')
        .when(col('mes') == 12, 'Dezembro')
    )
    .withColumn('eh_fim_de_semana', when(col('dia_da_semana_num').isin([1,7]), 'Sim').otherwise('Não'))
)

# for col_name in df_datas.columns:
#     if col_name != 'sk_tempo':
#         display(df_datas.select(col_name).distinct().orderBy(col_name))

df_datas.limit(20).display()

df_datas.write.mode('overwrite').saveAsTable(f'{catalogo}.{gold_db_name}.dm_tempo')

In [0]:
# 3.2 tabela gold.ft_vendas_geral

df_ft_itens_pedidos = spark.table(f'{catalogo}.{silver_db_name}.ft_itens_pedidos')
df_ft_pedidos = spark.table(f'{catalogo}.{silver_db_name}.ft_pedidos')
df_dm_cotacao = spark.table(f'{catalogo}.{silver_db_name}.dm_cotacao_dolar')
df_ft_avaliacoes_pedidos = spark.table(f'{catalogo}.{silver_db_name}.ft_avaliacoes_pedidos')

df_geral = (
    df_ft_itens_pedidos
    .join(df_ft_pedidos, on='id_pedido', how='left')
    .join(
        df_dm_cotacao,
        to_date(col('pedido_aprovado_timestamp')) == col('data'),
        how='left'
    )
    .join(df_ft_avaliacoes_pedidos, on='id_pedido', how='left')
    .select(
        col('id_pedido').cast('string'),
        col('id_item').cast('string'),
        col('id_consumidor').cast('string').alias('fk_cliente'),
        col('id_produto').cast('string').alias('fk_produto'),
        col('id_vendedor').cast('string').alias('fk_vendedor'),
        to_date(col('pedido_compra_timestamp')).alias('fk_tempo'),
        col('status').cast('string').alias('status_pedido'),
        col('tempo_entrega_dias').cast('int'),
        col('entrega_no_prazo').cast('string'),
        col('preco_BRL').cast(DecimalType(12,2)).alias('valor_produto_brl'),
        col('preco_frete').cast(DecimalType(12,2)).alias('valor_frete_brl'),
        (col('preco_BRL') + col('preco_frete')).cast(DecimalType(12,2)).alias('valor_total_item_brl'),
        (col('preco_BRL') / col('cotacao_dolar')).cast(DecimalType(12,2)).alias('valor_produto_usd'),
        (col('preco_frete') / col('cotacao_dolar')).cast(DecimalType(12,2)).alias('valor_frete_usd'),
        ((col('preco_BRL') + col('preco_frete')) / col('cotacao_dolar')).cast(DecimalType(12,2)).alias('valor_total_item_usd'),
        col('cotacao_dolar').cast(DecimalType(8,4)),
        col('avaliacao').cast(DecimalType(3,2)).alias('avaliacao_pedido')
    )
)

df_geral.limit(20).display()

df_geral.write.mode('overwrite').saveAsTable(f'{catalogo}.{gold_db_name}.ft_vendas_geral')

In [0]:
# 3.3 view gold.view_vendas_por_periodo

# levando em conta somente os pedidos entregues ou enviados 
query = """
CREATE OR REPLACE VIEW gold.view_vendas_por_periodo AS
SELECT
  t.ano,
  t.trimestre,
  t.mes,
  t.mes_nome,
  t.dia,
  t.dia_da_semana_num,
  COUNT(DISTINCT f.id_pedido) AS total_pedidos,
  COUNT(f.id_item) AS total_itens,
  SUM(f.valor_total_item_brl) AS receita_total_brl,
  SUM(f.valor_total_item_usd)::BIGINT AS receita_total_usd,
  ROUND(AVG(f.valor_total_item_brl), 2)::BIGINT AS ticket_medio_brl,
  CAST(ROUND(AVG(f.avaliacao_pedido), 2) AS DECIMAL(3,2)) AS avaliacao_media
FROM gold.ft_vendas_geral f
INNER JOIN gold.dm_tempo t
  ON f.fk_tempo = t.sk_tempo
WHERE f.status_pedido IN ('entregue', 'enviado')
GROUP BY
  t.ano,
  t.trimestre,
  t.mes,
  t.mes_nome,
  t.dia,
  t.dia_da_semana_num
ORDER BY
  total_pedidos ASC
"""

spark.sql(query)

display(spark.sql("SELECT * FROM gold.view_vendas_por_periodo"))

In [0]:
# 3.3.1.1. query dia da semana com maior receita total em reais
query_dia_semana_maior_receita = """
SELECT
  dia_da_semana_num,
  SUM(receita_total_brl) AS receita_total_brl
FROM gold.view_vendas_por_periodo
GROUP BY dia_da_semana_num
ORDER BY receita_total_brl DESC
LIMIT 1
"""

display(spark.sql(query_dia_semana_maior_receita))

In [0]:
# 3.3.1.2. query mes com maior ticket medio considerando o ultimo ano disponivel
query_mes_maior_ticket_ultimo_ano = """
WITH ultimo_ano AS (
  SELECT MAX(ano) AS ano
  FROM gold.view_vendas_por_periodo
)
SELECT
  mes,
  mes_nome,
  ROUND(MAX(ticket_medio_brl),2) AS ticket_medio_brl
FROM gold.view_vendas_por_periodo
WHERE ano = (SELECT ano FROM ultimo_ano)
GROUP BY mes, mes_nome
ORDER BY ticket_medio_brl DESC
LIMIT 1
"""

display(spark.sql(query_mes_maior_ticket_ultimo_ano))

In [0]:
# 3.4. view gold.view_top_produto

# levando em conta somente os pedidos entregues ou enviados 

query = """
CREATE OR REPLACE VIEW gold.view_top_produto AS
SELECT
  f.fk_produto AS id_produto,
  p.categoria_produto,
  SUM(1) AS quantidade_vendida,
  COUNT(DISTINCT f.id_pedido) AS total_pedidos,
  SUM(f.valor_total_item_brl) AS receita_brl,
  SUM(f.valor_total_item_usd) AS receita_usd,
  ROUND(AVG(f.valor_produto_brl), 2) AS preco_medio_brl,
  ROUND(AVG(f.avaliacao_pedido), 2) AS avaliacao_media,
  ROUND(AVG(p.peso_produto_gramas), 2) AS peso_medio_gramas
FROM gold.ft_vendas_geral f
LEFT JOIN silver.ft_produtos p
  ON f.fk_produto = p.id_produto
WHERE f.status_pedido IN ('entregue', 'enviado')
GROUP BY f.fk_produto, p.categoria_produto
ORDER BY receita_brl DESC, avaliacao_media DESC
"""

spark.sql(query)

display(spark.sql("SELECT * FROM gold.view_top_produto ORDER BY receita_brl DESC, avaliacao_media DESC LIMIT 20"))

In [0]:
# 3.5 view gold.view_vendas_produtos_esteticos

# levando em conta somente os pedidos entregues ou enviados 

query = """
CREATE OR REPLACE VIEW gold.view_vendas_produtos_esteticos AS
WITH vendas_esteticos AS (
  SELECT
    t.ano,
    t.mes,
    p.categoria_produto,
    f.id_pedido,
    f.id_item,
    f.valor_total_item_brl,
    f.valor_total_item_usd,
    f.avaliacao_pedido
  FROM gold.ft_vendas_geral f
  INNER JOIN gold.dm_tempo t
    ON f.fk_tempo = t.sk_tempo
  LEFT JOIN silver.ft_produtos p
    ON f.fk_produto = p.id_produto
  WHERE
    f.status_pedido IN ('entregue', 'enviado')
    AND p.categoria_produto LIKE 'fashion%'
)
SELECT
  ano,
  mes,
  categoria_produto,
  COUNT(DISTINCT id_pedido) AS total_pedidos,
  COUNT(id_item) AS total_itens_vendidos,
  ROUND(SUM(valor_total_item_brl), 2) AS receita_total_brl,
  ROUND(SUM(valor_total_item_usd), 2) AS receita_total_usd,
  ROUND(AVG(valor_total_item_brl), 2) AS ticket_medio_brl,
  ROUND(AVG(valor_total_item_usd), 2) AS ticket_medio_usd,
  ROUND(AVG(avaliacao_pedido), 2) AS avaliacao_media
FROM vendas_esteticos
GROUP BY ano, mes, categoria_produto
ORDER BY ano, mes, categoria_produto
"""

spark.sql(query)

display(spark.sql("SELECT * FROM gold.view_vendas_produtos_esteticos ORDER BY ano, mes, categoria_produto"))